<a href="https://colab.research.google.com/github/hufahamdika/BigData_Assignment9/blob/main/Collaborative_Filltering__5025201197.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b>Big Data Assignment Week 9</b>
### Collaborative Filtering
- Name : Abidjanna Zulfa Hamdika
- NRP : 5025201197
- Class : Big Data A

In [1]:
# Install Spark

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 22.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=9a09107b598a3589d2e8386fa783b12b3250b46fa0655bbee0fd132a71ae4f20
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
# Import Library

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [3]:
# Initialize Spark Session
spark = SparkSession.builder \
    .master("local") \
    .appName("MovieLens") \
    .getOrCreate()

spark.sparkContext.setCheckpointDir("./checkpoint")

In [5]:
# Read data from txt file and separate it for each line
lines = spark.read.text("./sample_data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))

In [6]:
# Convert data into a DataFrame with userId, movieId, rating, and timestamp columns and split it to train set (80%) and test set (%)
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [7]:
# Initialize the parameters and RMSE (Root Mean Squared Error) results
max_iters = [5, 10, 20]
reg_params = [0.1, 0.5, 1.0]

results = {}

In [8]:
# loop for every available parameters (max_iters and reg_params)
for max_iter in max_iters:
    for reg_param in reg_params:
        # Build the recommendation model using ALS on the training data
        # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        als = ALS(maxIter=max_iter, regParam=reg_param, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(training)

        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)

        # Save the RMSE result in the dictionary
        results[(max_iter, reg_param)] = rmse
        print(f"Root-mean-square error for maxIter={max_iter}, regParam={reg_param} = {rmse}")

Root-mean-square error for maxIter=5, regParam=0.1 = 1.0980025894305074
Root-mean-square error for maxIter=5, regParam=0.5 = 1.3403395705687726
Root-mean-square error for maxIter=5, regParam=1.0 = 1.601397865149919
Root-mean-square error for maxIter=10, regParam=0.1 = 1.085953856621306
Root-mean-square error for maxIter=10, regParam=0.5 = 1.33934295029582
Root-mean-square error for maxIter=10, regParam=1.0 = 1.6013987524399758
Root-mean-square error for maxIter=20, regParam=0.1 = 1.0999874878080267
Root-mean-square error for maxIter=20, regParam=0.5 = 1.3395420410940868
Root-mean-square error for maxIter=20, regParam=1.0 = 1.6013987530363245


In [9]:
# Find the the best hyperparameter by checking for the lowest RMSE
best_params = min(results, key=results.get)
best_rmse = results[best_params]
print(f"\nBest hyperparameters: maxIter={best_params[0]}, regParam={best_params[1]} with RMSE={best_rmse}")


Best hyperparameters: maxIter=10, regParam=0.1 with RMSE=1.085953856621306


In [10]:
# Train the model with the best hyperparameters
best_als = ALS(maxIter=best_params[0], regParam=best_params[1], userCol="userId", itemCol="movieId", ratingCol="rating",
               coldStartStrategy="drop")
best_model = best_als.fit(training)

In [11]:
# Top 10 movie recommendations for each users
userRecs = best_model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{22, 3.7739089},...|
|    10|[{92, 3.3430772},...|
|     0|[{92, 3.2897763},...|
|     1|[{62, 3.2322254},...|
|    21|[{29, 3.7919166},...|
|    11|[{27, 4.8759212},...|
|    12|[{46, 4.591204}, ...|
|    22|[{75, 4.6298304},...|
|     2|[{8, 4.5548162}, ...|
|    13|[{93, 3.1580074},...|
|     3|[{88, 3.278472}, ...|
|    23|[{32, 4.8106484},...|
|     4|[{62, 3.4732409},...|
|    24|[{69, 4.323935}, ...|
|    14|[{29, 4.72722}, {...|
|     5|[{55, 3.891803}, ...|
|    15|[{46, 3.8362877},...|
|    25|[{47, 3.2836604},...|
|    26|[{22, 4.573277}, ...|
|     6|[{29, 3.0228002},...|
+------+--------------------+
only showing top 20 rows



In [12]:
# Top 10 user recommendations for each movies
movieRecs = best_model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 3.9656518},...|
|     40|[{2, 3.5651405}, ...|
|     10|[{23, 3.2484283},...|
|     50|[{23, 3.9213653},...|
|     80|[{26, 3.3616567},...|
|     70|[{4, 3.0943582}, ...|
|     60|[{22, 2.490396}, ...|
|     90|[{17, 4.5779405},...|
|     30|[{22, 4.201868}, ...|
|      0|[{28, 2.3643532},...|
|     31|[{12, 2.8387053},...|
|     81|[{28, 4.0413823},...|
|     91|[{12, 2.6991887},...|
|      1|[{25, 2.1454346},...|
|     41|[{4, 3.2170773}, ...|
|     61|[{28, 1.8552811},...|
|     51|[{22, 4.168604}, ...|
|     21|[{26, 1.964947}, ...|
|     11|[{24, 1.0601054},...|
|     71|[{25, 3.146885}, ...|
+-------+--------------------+
only showing top 20 rows



In [13]:
# Generate top 10 movie recommendations for a specific set of users
users = ratings.select(best_als.getUserCol()).distinct().limit(3)
userSubsetRecs = best_model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{22, 4.573277}, ...|
|    19|[{32, 3.3303351},...|
|    29|[{46, 4.34456}, {...|
+------+--------------------+



In [14]:
# Generate top 10 user recommendations for a specific set of movies
movies = ratings.select(best_als.getItemCol()).distinct().limit(3)
movieSubSetRecs = best_model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{17, 1.7917507},...|
|     26|[{28, 2.7602966},...|
|     29|[{8, 4.982995}, {...|
+-------+--------------------+

